In [1]:
import pandas as pd
import os

def ler_dados_excel(caminho_arquivo, aba):
    # Ler o arquivo Excel usando o Pandas
    df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    return df

def analisar_dados(df, pessoa):
    # Filtrar apenas os dados da pessoa (PF ou PJ)
    df_pessoa = df[df['PESSOA'] == pessoa]

    # Número de beneficiários ativos
    num_beneficiarios_ativos = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['MATRICULA'].nunique()

    # Valor da receita
    valor_receita = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['Mensalidade'].sum()

    # Valor das despesas
    valor_despesas = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['Despesas'].sum()

    return num_beneficiarios_ativos, valor_receita, valor_despesas

def main():
    # Se estiver em um ambiente interativo, use o diretório atual como diretório do código
    if '__file__' not in globals():
        diretorio_codigo = os.getcwd()
    else:
        # Caso contrário, use o diretório do arquivo atual como diretório do código
        diretorio_codigo = os.path.dirname(os.path.abspath(__file__))

    # Caminho relativo para o arquivo analitixa-pf.xlsx dentro do diretório do código
    caminho_arquivo = os.path.join(diretorio_codigo, 'analitica.xlsx')

    abas = ['Janeiro-ANALITICO 2023', 'Fevereiro-ANALITICO 2023', 'Março-ANALITICO 2023',
            'Abril-ANALITICO 2023', 'Maio-ANALITICO 2023', 'Junho-ANALITICO 2023']

    dados_pf = []
    dados_pj = []

    for aba in abas:
        df = ler_dados_excel(caminho_arquivo, aba)

        if df is not None and not df.empty:
            num_beneficiarios_ativos_pf, valor_receita_pf, valor_despesas_pf = analisar_dados(df, 'PF')
            num_beneficiarios_ativos_pj, valor_receita_pj, valor_despesas_pj = analisar_dados(df, 'PJ')

            mes = aba.split('-')[0]

            dados_pf.append((mes, num_beneficiarios_ativos_pf, valor_receita_pf, valor_despesas_pf))
            dados_pj.append((mes, num_beneficiarios_ativos_pj, valor_receita_pj, valor_despesas_pj))

    # Criar a planilha com a aba "PF"
    df_pf = pd.DataFrame(dados_pf, columns=['MÊS', 'NÚMERO DE BENEFICIARIOS ATIVO PF', 'VALOR DA RECEITA PF',
                                            'VALOR DAS DESPESAS PF'])

    # Criar a planilha com a aba "PJ"
    df_pj = pd.DataFrame(dados_pj, columns=['MÊS', 'NÚMERO DE BENEFICIARIOS ATIVO PJ', 'VALOR DA RECEITA PJ',
                                            'VALOR DAS DESPESAS PJ'])

    # Exportar os DataFrames para um único arquivo Excel com várias abas
    nome_arquivo_saida = os.path.join(diretorio_codigo, 'planilhas_analise_pf_pj.xlsx')
    with pd.ExcelWriter(nome_arquivo_saida) as writer:
        df_pf.to_excel(writer, sheet_name='PF', index=False)
        df_pj.to_excel(writer, sheet_name='PJ', index=False)

if __name__ == '__main__':
    main()
